<div 
     style="padding: 20px; 
            color: yellow;
            margin: 0;
            font-size: 250%;
            text-align: center;
            display: fill;
            border-radius: 5px;
            background-color: #0daae3;
            overflow: hidden;
            font-weight: 700;
            border: 5px solid black;"
     >
            Candidate ReRank Model using Handcrafted Rules
</div>





## 导包

In [1]:
import os
import sys
import pickle 
import glob
import gc
import time

import itertools

import warnings
warnings.filterwarnings('ignore')

# from dask.distributed import Client
# from dask_cuda import LocalCUDACluster


# cluster = LocalCUDACluster() # by default use all GPUs in the node. I have two.
# client = Client(cluster)
os.environ['MODIN_ENGINE'] = 'dask'

import numpy as np
import modin.pandas as pd
from collections import Counter

In [2]:
path_project = r'/workspace/datasets/otto'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')

# path row_data
path_train = os.path.join(path_row_data, 'train.jsonl')
path_test  = os.path.join(path_row_data, 'test.jsonl')
path_sample_submission = os.path.join(path_row_data, 'sample_submission.csv')

# parquet 格式的文件存放路径
path_parquet = os.path.join(path_new_data, 'parquet')
path_parquet_train = os.path.join(path_parquet, 'train')
path_parquet_test = os.path.join(path_parquet, 'test')

path_top_40_carts_orders = os.path.join(path_new_data, 'top_40_carts_orders')
path_top_40_buy2buy = os.path.join(path_new_data, 'top_40_buy2buy')
path_top_40_clicks = os.path.join(path_new_data, 'top_40_clicks')

VERSION = 1

path_submission = os.path.join(path_results, 'version_{}'.format(VERSION))

In [3]:
files = sorted(glob.glob('{}/*'.format(path_parquet_train)))
chunk = int(np.ceil(len(files)/6))

print("{}个数据集将分为{}个部分进行处理".format(len(files), chunk))

129个数据集将分为22个部分进行处理


## Step 1 Candidate

### “点击/收藏/购买”共现矩阵-类别加权
假设用户当前浏览的商品会影响用户后续浏览商品的选择，因此构造商品-商品共现矩阵，其中用户对不同商品的动作代表的商品间的不同权重，因此可以通过这种逻辑来基于用户当前浏览的商品来预测下一个客户可能会浏览的商品
+ 这里用于当前商品的下一个浏览商品的推荐

In [4]:
%%time

type_labels = {"clicks":0, "carts":1, "orders":2} # 类别映射字典
type_weight = {0:1, 1:6, 2:3} # 权重映射字典

disk_pices = 4
size = 1.86e6/disk_pices

# 分块计算共现矩阵
for part in range(disk_pices):
    
    # 分块读取数据集
    for j in range(6):
        a = j*chunk
        b = min((j+1)*chunk, len(files))
        print("正在处理第{}-{}部分的数据".format(a, b))

        for k in range(a, b):
            # 读取数据
            df = pd.read_parquet(files[k])
            df.ts = (df.ts/1000).astype('int32')
            df['type'] = df['type'].map(type_labels).astype('int8')
            df = df.sort_values(['session', 'ts'], ascending=[True, False])
            # 对数量短的session进行过滤
            df = df.reset_index(drop=True)
            df['n'] = df.groupby('session').cumcount()
            df = df.loc[df.n<30].drop('n', axis=1)
            # 创建商品对
            df = df.merge(df, on='session')
            df = df.loc[((df.ts_x-df.ts_y).abs()<24*60*60)&(df.aid_x!=df.aid_y)]
            # 分步统计，节省内存
            df = df.loc[(df.aid_x >= part*size)&(df.aid_x < (part+1)*size)]
            # 分配权重
            df = df[['session', 'aid_x', 'aid_y', 'type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
            df['wgt'] = df.type_y.map(type_weight)
            df = df[['aid_x', 'aid_y', 'wgt']]
            df.wgt = df.wgt.astype('float32')
            df = df.groupby(['aid_x', 'aid_y']).wgt.sum()
            # 合并分块
            if k == a:
                tmp2 = df
            else:
                #tmp2 = pd.concat([tmp2, df]).filna(0)
                tmp2 = tmp2.add(df, fill_value=0)
            print(k, ' ', end='')
        print()
        if a==0: tmp = tmp2
        else:  tmp = tmp.add(tmp2, fill_value=0)  # tmp = pd.concat([tmp, tmp]).filna(0)            # tmp = tmp.add(tmp2, fill_value=0)
        del tmp2, df
        gc.collect()
    # 将矩阵转换为dict
    tmp = tmp.reset_index()
    tmp = tmp.sort_values(['aid_x', 'wgt'], ascending=[True, False])
    # 保存前40
    tmp = tmp.reset_index()
    tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
    tmp = tmp.loc[tmp.n<40].drop('n', axis=1)
    # 分块保存在磁盘中
    df = tmp.to_pandas().groupby('aid_x').aid_y.apply(list)
    os.makedirs(path_top_40_carts_orders, exist_ok=True)
    with open(os.path.join(path_top_40_carts_orders, 'top_40_carts_orders_v{}_{}.pkl'.format(VERSION, part)), 'wb') as f:
        pickle.dump(df.to_dict(), f)

正在处理第0-22部分的数据


2022-11-17 11:41:42,393 - distributed.worker_memory - WARNING - Worker tcp://127.0.0.1:34657 (pid=632) exceeded 95% memory budget. Restarting...
2022-11-17 11:41:42,429 - distributed.nanny - WARNING - Restarting worker


CancelledError: lambda-04b1ae3f9b1d082ce6433618f586d5f6

### “收藏/购买”共现矩阵
假设用户当前收藏或购买过的商品会影响用户后续的收藏/购买，因此构造商品-商品共现矩阵，其中这里假定收藏/购买的权重一致，因此可以通过这种逻辑来基于用户当前收藏/购买的商品来预测用户下一个可能会收藏/购买的商品。
+ 这里用户预测用户下一个可能收藏/购买的商品

In [5]:
%%time

type_labels = {"clicks":0, "carts":1, "orders":2} 

disk_pices = 16
size = 1.86e6/disk_pices

# 分块计算共现矩阵
for part in range(disk_pices):
    
    # 分块读取数据集
    for j in range(6):
        a = j*chunk
        b = min((j+1)*chunk, len(files))
        print("正在处理第{}-{}部分的数据".format(a, b))

        for k in range(a, b):
            # 读取数据
            df = cudf.read_parquet(files[k])
            df.ts = (df.ts/1000).astype('int32')
            df['type'] = df['type'].map(type_labels).astype('int8')
            df = df.loc[df['type'].isin([1,2])] # ONLY WANT CARTS AND ORDERS
            df = df.sort_values(['session', 'ts'], ascending=[True, False])
            # 对数量短的session进行过滤
            df = df.reset_index(drop=True)
            df['n'] = df.groupby('session').cumcount()
            df = df.loc[df.n<30].drop('n', axis=1)
            # 创建商品对
            df = df.merge(df, on='session')
            df = df.loc[((df.ts_x-df.ts_y).abs()<24*60*60)&(df.aid_x!=df.aid_y)]
            # 分步统计，节省内存
            df = df.loc[(df.aid_x >= part*size)&(df.aid_x < (part+1)*size)]
            # 分配权重
            df = df[['session', 'aid_x', 'aid_y', 'type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
            df['wgt'] = 1
            df = df[['aid_x', 'aid_y', 'wgt']]
            df.wgt = df.wgt.astype('float32')
            df = df.groupby(['aid_x', 'aid_y']).wgt.sum()
            # 合并分块
            if k == a: tmp2 = df
            else: tmp2 = tmp2.add(df, fill_value=0)
            print(k, ' ', end='')
        print()
        if a==0: tmp = tmp2
        else:  tmp = tmp.add(tmp2, fill_value=0)  
        gc.collect()
    # 将矩阵转换为dict
    tmp = tmp.reset_index()
    tmp = tmp.sort_values(['aid_x', 'wgt'], ascending=[True, False])
    # 保存前40
    tmp = tmp.reset_index()
    tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
    tmp = tmp.loc[tmp.n<40].drop('n', axis=1)
    # 分块保存在磁盘中
    df = tmp.to_pandas().groupby('aid_x').aid_y.apply(list)
    os.makedirs(path_top_40_buy2buy, exist_ok=True)
    with open(os.path.join(path_top_40_buy2buy, 'top_40_buy2buy_v{}_{}.pkl'.format(VERSION, part)), 'wb') as f:
        pickle.dump(df.to_dict(), f)

正在处理第0-22部分的数据
0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  
正在处理第22-44部分的数据
22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  
正在处理第44-66部分的数据
44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  
正在处理第66-88部分的数据
66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  
正在处理第88-110部分的数据
88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  
正在处理第110-129部分的数据
110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  
正在处理第0-22部分的数据
0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  
正在处理第22-44部分的数据
22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  
正在处理第44-66部分的数据
44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  
正在处理第66-88部分的数据
66  67  68  69  70  71  72  73  74  75  7

正在处理第110-129部分的数据
110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  
正在处理第0-22部分的数据
0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  
正在处理第22-44部分的数据
22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  
正在处理第44-66部分的数据
44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  
正在处理第66-88部分的数据
66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  
正在处理第88-110部分的数据
88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  
正在处理第110-129部分的数据
110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  
正在处理第0-22部分的数据
0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  
正在处理第22-44部分的数据
22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  
正在处理第44-66部分的数据
44  45  46  47  48  49  50  51  

### “点击/收藏/购买”共现矩阵-时间加权

In [6]:
%%time

type_labels = {"clicks":0, "carts":1, "orders":2} 

disk_pices = 16
size = 1.86e6/disk_pices

# 分块计算共现矩阵
for part in range(disk_pices):
    
    # 分块读取数据集
    for j in range(6):
        a = j*chunk
        b = min((j+1)*chunk, len(files))
        print("正在处理第{}-{}部分的数据".format(a, b))

        for k in range(a, b):
            # 读取数据
            df = cudf.read_parquet(files[k])
            df.ts = (df.ts/1000).astype('int32')
            df['type'] = df['type'].map(type_labels).astype('int8')
            df = df.loc[df['type'].isin([1,2])] # ONLY WANT CARTS AND ORDERS
            df = df.sort_values(['session', 'ts'], ascending=[True, False])
            # 对数量短的session进行过滤
            df = df.reset_index(drop=True)
            df['n'] = df.groupby('session').cumcount()
            df = df.loc[df.n<30].drop('n', axis=1)
            # 创建商品对
            df = df.merge(df, on='session')
            df = df.loc[((df.ts_x-df.ts_y).abs()<24*60*60)&(df.aid_x!=df.aid_y)]
            # 分步统计，节省内存
            df = df.loc[(df.aid_x >= part*size)&(df.aid_x < (part+1)*size)]
            # 分配权重
            df = df[['session', 'aid_x', 'aid_y', 'ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y'])
            df['wgt'] = 1 + 3*(df.ts_x - 1659304800)/(1662328791-1659304800)
            df = df[['aid_x', 'aid_y', 'wgt']]
            df.wgt = df.wgt.astype('float32')
            df = df.groupby(['aid_x', 'aid_y']).wgt.sum()
            # 合并分块
            if k == a: tmp2 = df
            else: tmp2 = tmp2.add(df, fill_value=0)
            print(k, ' ', end='')
        print()
        if a==0: tmp = tmp2
        else:  tmp = tmp.add(tmp2, fill_value=0)  
        gc.collect()
    # 将矩阵转换为dict
    tmp = tmp.reset_index()
    tmp = tmp.sort_values(['aid_x', 'wgt'], ascending=[True, False])
    # 保存前40
    tmp = tmp.reset_index()
    tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
    tmp = tmp.loc[tmp.n<40].drop('n', axis=1)
    # 分块保存在磁盘中
    df = tmp.to_pandas().groupby('aid_x').aid_y.apply(list)
    os.makedirs(path_top_40_clicks, exist_ok=True)
    with open(os.path.join(path_top_40_clicks, 'top_40_clicks_v{}_{}.pkl'.format(VERSION, part)), 'wb') as f:
        pickle.dump(df.to_dict(), f)

正在处理第0-22部分的数据
0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  
正在处理第22-44部分的数据
22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  
正在处理第44-66部分的数据
44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  
正在处理第66-88部分的数据
66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  
正在处理第88-110部分的数据
88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  
正在处理第110-129部分的数据
110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  
正在处理第0-22部分的数据
0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  
正在处理第22-44部分的数据
22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  
正在处理第44-66部分的数据
44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  
正在处理第66-88部分的数据
66  67  68  69  70  71  72  73  74  75  7

正在处理第110-129部分的数据
110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  
正在处理第0-22部分的数据
0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  
正在处理第22-44部分的数据
22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  
正在处理第44-66部分的数据
44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  
正在处理第66-88部分的数据
66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  
正在处理第88-110部分的数据
88  89  90  91  92  93  94  95  96  97  98  99  100  101  102  103  104  105  106  107  108  109  
正在处理第110-129部分的数据
110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  
正在处理第0-22部分的数据
0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  
正在处理第22-44部分的数据
22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  
正在处理第44-66部分的数据
44  45  46  47  48  49  50  51  

## Setp 2 ReRank

In [ ]:
def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob('{}/*'.format(path_parquet_test))):
        chunk = pd.read_parquet(chunk_file)
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True).astype({"ts": "datetime64[ms]"})

df_test = load_test()
print('Test data has shape',df_test.shape)
df_test.head()

In [ ]:
# 共现矩阵的结果

top_20_clicks = pickle.load(open(os.path.join(path_top_40_clicks,
                                'top_40_clicks_v{}_0.pkl'.format(VERSION)), 'rb'))
for k in range(1,disk_pices): 
    top_20_clicks.update(pickle.load(open(os.path.join(path_top_40_clicks,
                                'top_40_clicks_v{}_{}.pkl'.format(VERSION, k)), 'rb')))

top_20_buys = pickle.load(open(os.path.join(path_top_40_carts_orders,
                                'top_40_carts_orders_v{}_0.pkl'.format(VERSION)), 'rb'))
for k in range(1,disk_pices): 
    top_20_buys.update( pickle.load(open(os.path.join(path_top_40_carts_orders,
                                'top_40_carts_orders_v{}_{}.pkl'.format(VERSION, k)), 'rb')))

top_20_buy2buy = pickle.load(open(os.path.join(path_top_40_buy2buy,
                                'top_40_buy2buy_v{}_{}.pkl'.format(VERSION, k)), 'rb'))

# 测试集中的热门点击
top_clicks = df_test.loc[df_test['type']=='clicks','aid'].value_counts().index.values[:20]
top_orders = df_test.loc[df_test['type']=='orders','aid'].value_counts().index.values[:20]

print('共现矩阵的长度:')
len( top_20_clicks ), len( top_20_buy2buy ), len( top_20_buys )

In [ ]:
type_weight_multipliers = {'clicks': 1, 'carts': 6, 'orders': 3}

def suggest_clicks(df):
    # 用户历史浏览商品记录及类型
    aids = df.aid.tolist()
    types = df.type.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1]))
    # 通过权重进行重排序
    if len(unique_aids)>=20:
        # 时间加权
        weights = np.logspace(0.1, 1, len(aids), base=2, endpoint=True)-1
        aids_temp = Counter()
        # 使用商品出现次数及其类型进行重排序
        for aid, w, t in zip(aids, weights, types):
            aids_temp[aid] += w*type_weight_multipliers[t]
        sorted_aids = [k for k, v in aids_temp.most_common(20)]
        return sorted_aids
    # 使用“点击/收藏/购买”共现矩阵
    aids2 = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
    # 重排序
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2).most_common(20) if aid2 not in unique_aids]
    result = unique_aids + top_aids2[:20 - len(unique_aids)]
    # 使用测试集合的top20的点击
    return result + list(top_clicks)[:20-len(result)]

def suggest_buys(df):
    # 用户列浏览商品记录和类型
    aids = df.aid.tolist()
    types = df.type.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1]))
    df = df.loc[(df['type']=='carts')|df['type']=='orders']
    unique_buys = list(dict.fromkeys(df.aid.tolist()[::-1]))
    # 通过权重进行重排序
    if len(unique_aids)>=20:
        weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        # 使用buy2buy矩阵进行重排序
        aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
        for aid in aids3: aids_temp[aid] += 0.1
        sorted_aids = [k for k, v in aids_temp.most_common(20)]
        return sorted_aids
    # “点击/收藏/购买”共现矩阵
    aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
    # 使用buy2buy矩阵进行重排序
    aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
    # 重排序
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2+aids3).most_common(20) if aid2 not in unique_aids]
    result = unique_aids + top_aids2[:20-len(unique_aids)]
    
    return result + list(top_orders)[:20-len(result)]

## Submission

In [ ]:
%%time

pred_df_clicks = df_test.sort_values(["session", "ts"]).groupby(["session"]).apply(
    lambda x: suggest_clicks(x)
)

pred_df_buys = df_test.sort_values(["session", "ts"]).groupby(["session"]).apply(
    lambda x: suggest_buys(x)
)

In [ ]:
clicks_pred_df = pd.DataFrame(pred_df_clicks.add_suffix("_clicks"), columns=["labels"]).reset_index()
orders_pred_df = pd.DataFrame(pred_df_buys.add_suffix("_orders"), columns=["labels"]).reset_index()
carts_pred_df = pd.DataFrame(pred_df_buys.add_suffix("_carts"), columns=["labels"]).reset_index()

In [ ]:
pred_df = pd.concat([clicks_pred_df, orders_pred_df, carts_pred_df])
pred_df.columns = ["session_type", "labels"]
pred_df["labels"] = pred_df.labels.apply(lambda x: " ".join(map(str,x)))

os.makedirs(path_submission, exist_ok=True)
pred_df.to_csv(os.path.join(path_submission, time.strftime('submission_%Y%m%d%H%M.csv')), index=False)
pred_df.head()